In [41]:
# Classification And Regression Trees 回归树
# https://blog.csdn.net/jiaoyangwm/article/details/79525237

# https://github.com/apachecn/AiLearning/blob/master/docs/ml/9.%E6%A0%91%E5%9B%9E%E5%BD%92.md
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree

import numpy as np
from math import log
import operator

In [82]:
# 年龄：0代表青年，1代表中年，2代表老年；
# 有工作：0代表否，1代表是；
# 有自己的房子：0代表否，1代表是；
# 信贷情况：0代表一般，1代表好，2代表非常好；
# 类别(是否给贷款)：no代表否，yes代表是。
# labels=['年龄','有工作','有自己的房子','信贷情况']
dataSet = [[0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0],
           [0, 1, 0, 1, 1],
           [0, 1, 1, 0, 1],
           [0, 0, 0, 0, 0],
           [1, 0, 0, 0, 0],
           [1, 0, 0, 1, 0],
           [1, 1, 1, 1, 1],
           [1, 0, 1, 2, 1],
           [1, 0, 1, 2, 1],
           [2, 0, 1, 2, 1],
           [2, 0, 1, 1, 1],
           [2, 1, 0, 1, 1],
           [2, 1, 0, 2, 1],
           [2, 0, 0, 0, 0]]

In [83]:
# 信息熵(information entropy)
def calcShannonEnt(dataSet):
    #返回数据集行数
    numEntries=len(dataSet)
    #保存每个标签（label）出现次数的字典
    labelCounts={}
    #对每组特征向量进行统计
    for featVec in dataSet:
        currentLabel=featVec[-1]                     #提取标签信息
        if currentLabel not in labelCounts.keys():   #如果标签没有放入统计次数的字典，添加进去
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1                 #label计数
    print(labelCounts)
    shannonEnt=0.0
    #计算信息熵
    for key in labelCounts:
        prob=float(labelCounts[key])/numEntries      #选择该标签的概率
        shannonEnt-=prob*log(prob,2)                 #利用公式计算
    return shannonEnt

calcShannonEnt(dataSet)

{0: 6, 1: 9}


0.9709505944546686

In [84]:
# 信息增益(information gain)

def chooseBestFeatureToSplit(dataSet):
    #特征数量
    numFeatures = len(dataSet[0]) - 1
    #计数数据集的香农熵
    baseEntropy = calcShannonEnt(dataSet)
    #信息增益
    bestInfoGain = 0.0
    #最优特征的索引值
    bestFeature = -1
    #遍历所有特征
    for i in range(numFeatures):
        # 获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        #创建set集合{}，元素不可重复
        uniqueVals = set(featList)
        #经验条件熵
        newEntropy = 0.0
        #计算信息增益
        for value in uniqueVals:
            #subDataSet划分后的子集
            subDataSet = splitDataSet(dataSet, i, value)
            #计算子集的概率
            prob = len(subDataSet) / float(len(dataSet))
            #根据公式计算经验条件熵
            newEntropy += prob * calcShannonEnt((subDataSet))
        #信息增益
        infoGain = baseEntropy - newEntropy
        #打印每个特征的信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))
        #计算信息增益
        if (infoGain > bestInfoGain):
            #更新信息增益，找到最大的信息增益
            bestInfoGain = infoGain
            #记录信息增益最大的特征的索引值
            bestFeature = i
            #返回信息增益最大特征的索引值
    return bestFeature

"""
Parameters：
    dataSet：待划分的数据集
    axis：划分数据集的特征
    value：需要返回的特征的值

"""
def splitDataSet(dataSet,axis,value):
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis]==value:
            reducedFeatVec=featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

chooseBestFeatureToSplit(dataSet)

{0: 6, 1: 9}
{0: 3, 1: 2}
{0: 2, 1: 3}
{1: 4, 0: 1}
第0个特征的增益为0.083
{0: 6, 1: 4}
{1: 5}
第1个特征的增益为0.324
{0: 6, 1: 3}
{1: 6}
第2个特征的增益为0.420
{0: 4, 1: 1}
{0: 2, 1: 4}
{1: 4}
第3个特征的增益为0.363


2

In [96]:
"""
函数说明：统计classList中出现次数最多的元素（类标签）
Parameters：
    classList：类标签列表
Returns：
    sortedClassCount[0][0]：出现次数最多的元素（类标签）
Modify：
    2018-03-13

"""
def majorityCnt(classList):
    classCount={}
    #统计classList中每个元素出现的次数
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
            classCount[vote]+=1
        #根据字典的值降序排列
        sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
        return sortedClassCount[0][0]
    
def createTree(dataSet,labels,featLabels):
    #取分类标签（是否放贷：yes or no）
    classList=[example[-1] for example in dataSet]
    #如果类别完全相同，则停止继续划分
    if classList.count(classList[0])==len(classList):
        return classList[0]
    #遍历完所有特征时返回出现次数最多的类标签
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    #选择最优特征
    bestFeat=chooseBestFeatureToSplit(dataSet)
    #最优特征的标签
    bestFeatLabel=labels[bestFeat]
    featLabels.append(bestFeatLabel)
    #根据最优特征的标签生成树
    myTree={bestFeatLabel:{}}
    #删除已经使用的特征标签
    del(labels[bestFeat])
    #得到训练集中所有最优特征的属性值
    featValues=[example[bestFeat] for example in dataSet]
    #去掉重复的属性值
    uniqueVls=set(featValues)
    #遍历特征，创建决策树
    for value in uniqueVls:
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),labels,featLabels)
    return myTree

def predict(inputTree,featLabels,testVec):
    #获取决策树节点
    firstStr=next(iter(inputTree))
    print(firstStr)
    #下一个字典
    secondDict=inputTree[firstStr]
    print(secondDict)
    featIndex=featLabels.index(firstStr)
    print(featIndex)
    print('keys: ',secondDict.keys())

    for key in secondDict.keys():
        if testVec[featIndex]==key:
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],featLabels,testVec)
            else: classLabel=secondDict[key]
    return classLabel

In [97]:
labels=['年龄','有工作','有自己的房子','信贷情况']
featLabels=[]
myTree=createTree(dataSet,labels,featLabels)
print(myTree)

#测试数据
testVec=[0,1]
result=predict(myTree,featLabels,testVec)
if result:
    print('放贷')
else:
    print('不放贷')

{0: 6, 1: 9}
{0: 3, 1: 2}
{0: 2, 1: 3}
{1: 4, 0: 1}
第0个特征的增益为0.083
{0: 6, 1: 4}
{1: 5}
第1个特征的增益为0.324
{0: 6, 1: 3}
{1: 6}
第2个特征的增益为0.420
{0: 4, 1: 1}
{0: 2, 1: 4}
{1: 4}
第3个特征的增益为0.363
{0: 6, 1: 3}
{0: 3, 1: 1}
{0: 2}
{1: 2, 0: 1}
第0个特征的增益为0.252
{0: 6}
{1: 3}
第1个特征的增益为0.918
{0: 4}
{0: 2, 1: 2}
{1: 1}
第2个特征的增益为0.474
{'有自己的房子': {0: {'有工作': {0: 0, 1: 1}}, 1: 1}}
有自己的房子
{0: {'有工作': {0: 0, 1: 1}}, 1: 1}
0
keys:  dict_keys([0, 1])
有工作
{0: 0, 1: 1}
1
keys:  dict_keys([0, 1])
放贷


In [ ]:
# https://github.com/ljpzzz/machinelearning/blob/master/classic-machine-learning/decision_tree_classifier.ipynb